# COMP0189: Applied Artificial Intelligence
## Week 3 (Model Selection and Assessment)

### After this week you will be able to ...
- encode categorical values with one-hot encoding
- know which encoding, scaling, and imputing method you should select in accordacne with the dataset characteristics
- impute missing data with KNN
- know how to streamline the preprocessing steps in advanced way (Pipeline and ColmnTransformer)
- select best model based on various cross-validation methods

### Acknowledgements
- https://scikit-learn.org/stable/
- https://archive.ics.uci.edu/ml/datasets/adult

In [ ]:
# Optional: For black-style code formatting within the notebook env.
!pip install nb-black

In [ ]:
# Optional: For black-style code formatting within the notebook env.
%load_ext nb_black

In [ ]:
# Previously, we used version 1.1.3 to use boston dataset as an example.
# Since there has been many updates in core functions after version 1.2,
# we are updating the scikit-learn to the latest version.
# Otherwise you will get some errors and FutureWarnings.
!pip install scikit-learn==1.2.1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Part 1: Encoding and Imputations

### Task 1: Load and Split the Dataset into train and test

In [ ]:
# TASK 1: Load Dataset
# We are going to use the same adult dataset as previous week.
# We have cleaned the dataset, but did not touch the missing values.
df = pd.read_csv("../data/clean_adult.csv")
df

In [ ]:
sorted(list(df["Education-num"].unique()))

In [ ]:
len(df["Education"].unique())

In [ ]:
# We can see that Education-num is just a label-encoded feature of Education
edu_map = {}
for i, row in df[["Education", "Education-num"]].iterrows():
    education = row["Education"]
    edu_num = row["Education-num"]

    if education not in edu_map:
        edu_map.update({education: edu_num})
    else:
        assert edu_map[education] == edu_num
edu_map

In [ ]:
# let's just drop Education column as we don't need it if we have Education-num
df = df.drop(["Education"], axis=1)
df

In [ ]:
def train_test_split_df(df, test_ratio=0.1, target_col="Y"):
    df_data = df.drop([target_col], axis=1)
    df_target = df[target_col]

    split_index = int(len(df) * (1 - test_ratio))
    print(f"Splitting from index {split_index}")

    train_X_df = df_data[:split_index]
    test_X_df = df_data[split_index:]
    train_y_df = df_target[:split_index]
    test_y_df = df_target[split_index:]

    train_y_df = np.where(train_y_df == ">50K", 1, 0)
    test_y_df = np.where(test_y_df == ">50K", 1, 0)

    return train_X_df, train_y_df, test_X_df, test_y_df

In [ ]:
# Splitting dataset into train and test
train_X_df, train_y_df, test_X_df, test_y_df = train_test_split_df(df)

In [ ]:
# Just for educational purpose:
# For this dataset split, you can see that
# all the unique values of categorical features in test set are subset of that in train set.
# However, we shouldn't expect this, thus need to deal with this situation (dealt in later cells).
for col in train_X_df:
    # Skips for the non-categorical values
    if col in [
        "Age",
        "Fnlwgt",
        "Capital-gain",
        "Capital-loss",
        "Hours-per-week",
    ]:
        continue
    train_uniq_vals = train_X_df[col].unique()
    test_uniq_vals = test_X_df[col].unique()

    if not set(test_uniq_vals).issubset(set(train_uniq_vals)):
        print(col)
        print(set(test_uniq_vals))
        print(set(train_uniq_vals))

### Task 2: Encode categorical variables (label/ordinal encoding & one-hot encoding)

### Important: We need special care when we are encoding categorical variables 

**1. Take care of the missing values**
- Beware not to encode missing values unless you are intending to do so.
- Sometimes you want to encode missing values to a separate cateogory. For example, when you want to predict if passengers of titanic had survived or not, missing data of certain features can actually have meaning, i.e., Cabin information can be missing because the body was not found.

**2. Know which encoding and scaling method you should select**
- If your categories are ordinal, then it makes sense to use a LabelEncoder with a MinMaxScaler. For example, you can encode [low, medium, high], as [1,2,3], i.e., distance between low to high is larger than that of medium and high.

- However, if you have non-ordinal categorical values, like [White, Hispanic, Black, Asian], then it would be better to use a OneHotEncoder instead of forcing ordinality with a LabelEncoder. Otherwise the algorithms you use (especially distance based algorithms like KNN) will make the assumption that the distance between White and Asian is larger than White and Hispanic, which is nonsensical.

**3. Split before you encode to avoid data leakage**
- Split the dataset before you encode your data. It is natural for algorithms to see unknown values in the validation/test set that was not appearing in the train set. `sklearn.preprocessing.OneHotEncoder` is good at handling these unknown categories (`handle_unknown` parameter).

- Discussion: What if you are certain about all the possible categories that can appear for each feature? Can you encode all the values before splitting the dataset into train and test set?


This notebook shows the three points in the following sections with examples.

### Task 2-1: Label Encoding (with missing values)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

label_encoder = LabelEncoder()

In [ ]:
train_X_df.isnull().sum()

In [ ]:
mask_df = train_X_df.isnull()
# We will encode these columns with LabelEncoder, and the rest with OneHotEncoder
categ = ["Sex"]
train_X_df[categ] = train_X_df[categ].apply(label_encoder.fit_transform)
# This masking process won't give any effect in this cell, where 'SEX' and 'Y' don't have any missing values.
# However, if the columns we are encoding have missing values,
# this is how you avoid encoding missing values as a separate category.
# To see the effect of masking, add 'Workclass' in the categ variable and check df.isnull().sum().
train_X_df = train_X_df.mask(mask_df, np.nan)
train_X_df

In [ ]:
# The missing values are intact and were not encoded.
train_X_df.isnull().sum()

In [ ]:
train_X_df["Race"].unique()

### Task 2-2: One Hot Encoding (with missing values imputation)

Tip 1: Impute the missing values (choose the right strategy) before doing OHE  
Tip 2: Try creating a separate dataframe with one-hot encoded columns and combine the dataframe with the original dataframe for the final one.

In [ ]:
# Let's first impute the missing values.
# Since it's a categorical value, we don't use KNN or mean imputation.
# We will replace with the most frequent value.
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")
imputed_train_X = imputer.fit_transform(train_X_df)
imputed_train_X_df = pd.DataFrame(imputed_train_X, columns=train_X_df.columns)

# Check that we have no missing values now
imputed_train_X_df.isnull().sum()

In [ ]:
# We want to turn these features into one-hot vectors
onehot_categ = [
    "Workclass",
    "Marital-status",
    "Occupation",
    "Relationship",
    "Race",
    "Native-country",
]
onehot_encoder = OneHotEncoder(sparse_output=False).fit(
    imputed_train_X_df[onehot_categ]
)
encoded = onehot_encoder.transform(imputed_train_X_df[onehot_categ])
encoded_df = pd.DataFrame(encoded, columns=onehot_encoder.get_feature_names_out())
encoded_df

In [ ]:
# After finishing encoding categorical variables,
# we make the final dataframe by concatenating it with the imputed dataframe
imputed_train_X_df = imputed_train_X_df.drop(onehot_categ, axis=1)
final_df_train_X_df = pd.concat([imputed_train_X_df, encoded_df], axis=1)
final_df_train_X_df

In [ ]:
final_df_train_X_df.shape

### Another (recommended) Method: Using `Pipeline` and `ColmnTransformer` to streamline the workflow.

We will do the same operation (with addition of scaling) but with much more succinct way.   
We use `Pipeline` and `ColmnTransformer`.  
`ColmnTransformer` is needed to apply different preprocessing steps to selected columns.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler

In [ ]:
# Reload the dataset
df = pd.read_csv("../data/clean_adult.csv")
df = df.drop(["Education"], axis=1)
train_X, train_y, test_X, test_y = train_test_split_df(df)

In [ ]:
non_categorical_features = [
    "Age",
    "Fnlwgt",
    "Capital-gain",
    "Capital-loss",
    "Hours-per-week",
]
categorical_ohe_features = [
    "Workclass",
    "Education-num",
    "Marital-status",
    "Occupation",
    "Relationship",
    "Race",
    "Native-country",
]
categorical_le_features = ["Sex"]

In [ ]:
# For features like 'Age' and 'Fnlwgt'
non_categorical_transformer = Pipeline(
    # For KNNImputer, see the side note below
    # We can add scaling for non-categorical features
    steps=[("KNNImputer", KNNImputer(n_neighbors=5)), ("scaling", StandardScaler())]
)

# For features like 'Workclass' and 'Education'
categorical_ohe_transformer = Pipeline(
    steps=[
        ("SimpleImputer", SimpleImputer(strategy="most_frequent")),
        ("OHE", OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
        # no need to scale
    ]
)

# For features like 'Sex'
categorical_le_transformer = Pipeline(
    steps=[
        ("ModeImputer", SimpleImputer(strategy="most_frequent")),
        # Here, we change to the OrdinalEncoder as LabelEncoder is used for the target variable
        # Try changing the OrdinalEncoder to LabelEncoder to see what error you see,
        # and check the documentation of LabelEncoder
        ("LE", OrdinalEncoder()),
        # In the case of adult dataset, no need to scale for just 'Sex' variable,
        # but other label/ordinal encoded categorical features can be ordinal -> scaling
        ("scaling", StandardScaler()),
    ]
)

In [ ]:
ct = ColumnTransformer(
    transformers=[
        ("non-categorical", non_categorical_transformer, non_categorical_features),
        ("categorical-ohe", categorical_ohe_transformer, categorical_ohe_features),
        ("categorical-le", categorical_le_transformer, categorical_le_features),
    ]
)

In [ ]:
transformed_train_X = ct.fit_transform(train_X, train_y)
transformed_test_X = ct.transform(test_X)

In [ ]:
transformed_train_X.shape, train_y.shape, transformed_test_X.shape, test_y.shape

### Side Note: Data Imputation with KNN
For the adult dataset, missing data present only in categorical values, so imputing strategy that makes floating point may not make sense.
However, for continuous values, you can use various imputation strategies, such as taking simple mean or mean value from K nearest neighbors (KNN).
If you use `sklearn.imput.KNNImputer`, each sample’s missing values are imputed using the `mean` value from `n_neighbors` nearest neighbors found in the training set.
If you want to use `mode` value from neighbors (for categorical data imputation) you need to implement the imputer by yourself.

- `sklearn-pandas` package (https://pypi.org/project/sklearn-pandas/1.5.0/) provides `CategoricalImputer` class, which is suitable for such processing

Here, we use iris dataset to show how to use KNNImputer for continuous values

In [ ]:
from sklearn.datasets import load_iris
from sklearn.impute import KNNImputer

In [ ]:
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [ ]:
# Applying a random mask to make missing data
mask = np.random.choice([True, False], size=iris_df.shape[0] * iris_df.shape[1])
mask[:500] = True
np.random.shuffle(mask)
mask = np.reshape(mask, iris_df.shape)
iris_df = iris_df.mask(~mask)

iris_df.isnull().sum()

In [ ]:
train_X, test_X = iris_df[:100], iris_df[100:]

In [ ]:
# It is important to impute the train and test set separately (not fitting KNN to test set) to avoid data leak.
imputer = KNNImputer(n_neighbors=5)
imputed_train_X = imputer.fit_transform(train_X)
imputed_test_X = imputer.transform(test_X)

In [ ]:
del iris, iris_df, mask, train_X, test_X, imputer, imputed_train_X, imputed_test_X

### Task 3: Create different preprocessing strategies of your own
Create different versions of X (X1 and X2) and y using different strategies for data imputation.  
Define different preprocessing strategies using the `Pipeline` and `ColmnTransformer` class

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
# Your explorations here

### Task 4: 
Train different models (KNN, SVM) to predict the y from the two versions of X (X1 and X2) with a fixed value of the regularization parameter. 
Centre and scale the data before training the models. Create tables or plots to show how accuracy varies for different imputation strategies or different models. 

### Task 4-1: Training KNN and SVM Models (with original preprocessing)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import mutual_info_classif

In [ ]:
model_knn = KNeighborsClassifier()
model_knn.fit(transformed_train_X, train_y)

# Predict Output
y_hat_knn = model_knn.predict(transformed_test_X)
accuracy_score(y_hat_knn, test_y)

In [ ]:
model_svm = svm.SVC(kernel="linear")  # Linear Kernel
model_svm.fit(transformed_train_X, train_y)

# Predict Output
y_hat_svm = model_svm.predict(transformed_test_X)
accuracy_score(y_hat_svm, test_y)

### Task 4-2: Investigation
Create tables or plots to show how accuracy varies for different imputation strategies or different models. 
- What is the impact of the imputation strategy on the accuracy? 
- What is the impact of the model on the accuracy? 

## Part 2: Cross Validation (CV)

`scikit-learn` provides a nice visulaisation of various cross validation methods.  
This notebook focuses on the three main CV techniques: `KFold`, `StratifiedKFold`, `GroupKFold`, and `StratifiedGroupKFold` for optimizing models' hyper-parameters and to understand how different strategies might affect the models' performance. 

Visit: https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#visualizing-cross-validation-behavior-in-scikit-learn

![kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_006.png)
![stra-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_003.png)
![group-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_004.png)
![stra-group-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_010.png)

In [ ]:
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    GroupKFold,
    StratifiedGroupKFold,
    GridSearchCV,
)

### Task 5
Now do CV to optimize the models hyperparameter using different k (2, 5, 10, 20) for the k-fold CV (KNN, SVM) to predict the y from the two versions of X (X1 and X2) and plot the model performance (mean accuracy and SD) for the different k. Centre and scale the data before training the models. 

Note: remember that the pre-processing steps, including data centering and scaling should be embedded in the CV. 

- How does the accuracy vary as the number of folds increase? 

In [ ]:
result = []
for n in [3,5,10,20]:
    kf = KFold(n_splits=n)
    tem = []
    for i, (train_index, test_index) in enumerate(kf.split(transformed_train_X)):
        model_knn = KNeighborsClassifier()
        k_range = np.arange(1, 10)
        # define grid search
        grid = dict(n_neighbors=k_range)
        cv = KFold(n_splits=n-1)
        grid_search = GridSearchCV(
        estimator=model_knn,
        param_grid=grid,
        pre_dispatch = 6,
        n_jobs=6,
        cv=cv,
        scoring="accuracy",
        error_score=0,
        verbose=1,
        )
        grid_result = grid_search.fit(transformed_train_X[train_index], train_y[train_index])
        model_knn = KNeighborsClassifier(n_neighbors = grid_result.best_params_["n_neighbors"])
        model_knn.fit(transformed_train_X[train_index], train_y[train_index])


        y_hat_knn = model_knn.predict(transformed_train_X[test_index])
        tem.append(accuracy_score(y_hat_knn, train_y[test_index]))
    result.append(tem)


In [ ]:
result = []
for n in [3,5,10,20]:
    kf = KFold(n_splits=n)
    tem = []
    for i, (train_index, test_index) in enumerate(kf.split(transformed_train_X)):
        model_svm = svm.SVC(kernel="linear")
        C_list = [0.1, 0.5, 1, 5]
        # define grid search
        grid = dict(C=C_list)
        cv = KFold(n_splits=n-1)
        grid_search = GridSearchCV(
        estimator=model_svm,
        param_grid=grid,
        pre_dispatch = 16,
        n_jobs=16,
        cv=cv,
        scoring="accuracy",
        error_score=0,
        verbose=1,
        )
        grid_result = grid_search.fit(transformed_train_X[train_index], train_y[train_index])
        model_svm = svm.SVC(kernel="linear",C=grid_result.best_params_["C"])
        model_svm.fit(transformed_train_X[train_index], train_y[train_index])
        y_hat_svm = model_svm.predict(transformed_train_X[test_index])
        tem.append(accuracy_score(y_hat_svm, train_y[test_index]))
    result.append(tem)


### Task 6
Repeat task 5 using stratified CV with k=5. Compute the accuracy of the models implemented in task 5 (with k=5) and check if the model with stratified CV performs better across the different folds. Centre and scale the data before training the models. Create tables or plots to show these results.

In [ ]:
n = 5
kf = StratifiedKFold(n_splits=n)
tem = []
for i, (train_index, test_index) in enumerate(kf.split(transformed_train_X, train_y)):
    model_knn = KNeighborsClassifier()
    k_range = np.arange(1, 10)
    # define grid search
    grid = dict(n_neighbors=k_range)
    cv = StratifiedKFold(n_splits=n - 1)
    grid_search = GridSearchCV(
        estimator=model_knn,
        param_grid=grid,
        n_jobs=2,
        cv=cv,
        scoring="accuracy",
        error_score=0,
        verbose=1,
    )
    grid_result = grid_search.fit(
        transformed_train_X[train_index], train_y[train_index]
    )
    model_knn = KNeighborsClassifier(
        n_neighbors=grid_result.best_params_["n_neighbors"]
    )
    model_knn.fit(transformed_train_X[train_index], train_y[train_index])

    y_hat_knn = model_knn.predict(transformed_train_X[test_index])
    tem.append(accuracy_score(y_hat_knn, train_y[test_index]))

In [ ]:
tem

In [ ]:
kf = StratifiedKFold(n_splits=n)
tem = []
for i, (train_index, test_index) in enumerate(kf.split(transformed_train_X, train_y)):
    model_svm = svm.SVC(kernel="linear")
    C_list = [0.1, 0.5, 1, 5]
    # define grid search
    grid = dict(C=C_list)
    cv = StratifiedKFold(n_splits=n - 1)
    grid_search = GridSearchCV(
        estimator=model_svm,
        param_grid=grid,
        n_jobs=2,
        cv=cv,
        scoring="accuracy",
        error_score=0,
        verbose=1,
    )
    grid_result = grid_search.fit(
        transformed_train_X[train_index], train_y[train_index]
    )
    model_svm = svm.SVC(kernel="linear", C=grid_result.best_params_["C"])
    model_svm.fit(transformed_train_X[train_index], train_y[train_index])
    y_hat_svm = model_svm.predict(transformed_train_X[test_index])
    tem.append(accuracy_score(y_hat_svm, train_y[test_index]))

### Task 7
Repeat task 5 using stratified group CV considering 'Race' as a group with k=5.
Compute the accuracy of the models implemented in task 5 (with k=5) and check if the model with stratified group CV performs better across the different races. 
Centre and scale the data before training the models. 
Create tables or plots to show these results. 

In [ ]:
n = 5
kf = StratifiedGroupKFold(n_splits=n)
tem = []
group = train_X_df["Race"]
for i, (train_index, test_index) in enumerate(
    kf.split(transformed_train_X, train_y, group)
):
    model_knn = KNeighborsClassifier()
    k_range = np.arange(1, 10)
    # define grid search
    grid = dict(n_neighbors=k_range)
    tem_group = train_X_df["Race"][train_index]
    cv = StratifiedGroupKFold(n_splits=n - 1)
    grid_search = GridSearchCV(
        estimator=model_knn,
        param_grid=grid,
        n_jobs=2,
        cv=cv,
        scoring="accuracy",
        error_score=0,
        verbose=1,
    )
    grid_result = grid_search.fit(
        transformed_train_X[train_index], train_y[train_index], groups=tem_group
    )
    model_knn = KNeighborsClassifier(
        n_neighbors=grid_result.best_params_["n_neighbors"]
    )
    model_knn.fit(transformed_train_X[train_index], train_y[train_index])

    y_hat_knn = model_knn.predict(transformed_train_X[test_index])
    tem.append(accuracy_score(y_hat_knn, train_y[test_index]))

In [ ]:
tem

In [ ]:
kf = StratifiedGroupKFold(n_splits=n)
tem = []
group = train_X_df["Race"]
for i, (train_index, test_index) in enumerate(kf.split(transformed_train_X, train_y,group)):
    model_svm = svm.SVC(kernel="linear")
    C_list = [0.1, 0.5, 1, 5]
    # define grid search
    grid = dict(C=C_list)
    tem_group = train_X_df["Race"][train_index]
    cv = StratifiedGroupKFold(n_splits=n - 1)
    grid_search = GridSearchCV(
        estimator=model_svm,
        param_grid=grid,
        n_jobs=2,
        cv=cv,
        scoring="accuracy",
        error_score=0,
        verbose=1,
    )
    grid_result = grid_search.fit(
        transformed_train_X[train_index], train_y[train_index],groups=tem_group
    )
    model_svm = svm.SVC(kernel="linear", C=grid_result.best_params_["C"])
    model_svm.fit(transformed_train_X[train_index], train_y[train_index])
    y_hat_svm = model_svm.predict(transformed_train_X[test_index])
    tem.append(accuracy_score(y_hat_svm, train_y[test_index]))

In [ ]:
tem